In [2]:
!git clone https://github.com/hunglc007/tensorflow-yolov4-tflite.git

fatal: destination path 'tensorflow-yolov4-tflite' already exists and is not an empty directory.


In [4]:
%cd tensorflow-yolov4-tflite

/kaggle/working/tensorflow-yolov4-tflite


In [9]:
!pip install -r requirements-gpu.txt

     |████████████████████████████████| 320.4 MB 8.6 kB/s  eta 0:00:01    |█████▉                          | 57.9 MB 39.7 MB/s eta 0:00:07     |██████▎                         | 63.3 MB 17.4 MB/s eta 0:00:15     |██████▍                         | 64.2 MB 17.4 MB/s eta 0:00:15     |██████▌                         | 64.9 MB 17.4 MB/s eta 0:00:15     |██████▋                         | 65.8 MB 17.4 MB/s eta 0:00:15     |██████▋                         | 66.7 MB 17.4 MB/s eta 0:00:15     |██████▉                         | 68.4 MB 17.4 MB/s eta 0:00:15     |█████████▍                      | 94.2 MB 35.3 MB/s eta 0:00:07     |█████████████████████████▊      | 257.5 MB 37.6 MB/s eta 0:00:02
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.4.1
    Uninstalling tensorflow-gpu-2.4.1:
      Successfully uninstalled tensorflow-gpu-2.4.1


---
# need to change class names in config.py

In [17]:
# current dir = output/tensorflow-yolov4-tflite
!cp ../../input/yolo-tflite-jackpodkim/obj.names ./data/classes/
!ls ./data/classes/

coco.names  obj.names  voc.names  yymnist.names


In [18]:
!sed -i "s/coco.names/obj.names/g" ./core/config.py

---
# convert

In [28]:
#../input/yolo-tflite-jackpodkim/yolov4_tiny_custom_best.weights
!python save_model.py\
 --weights ../../input/yolo-tflite-jackpodkim/yolov4_tiny_custom_best.weights\
 --output ./yolov4-tiny-416\
 --input_size 416\
 --model yolov4\
 --tiny

2021-08-03 11:37:19.811223: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64:/opt/conda/lib
2021-08-03 11:37:19.811309: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-03 11:37:22.005626: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-03 11:37:22.023304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-03 11:37:22.024534: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB comp

In [29]:
# SavedModel to convert to TFLite
!python save_model.py \
  --weights ../../input/yolo-tflite-jackpodkim/yolov4_tiny_custom_best.weights \
  --output ./yolov4-tiny-pretflite-416 \
  --input_size 416 \
  --model yolov4 \
  --tiny \
  --framework tflite

2021-08-03 11:37:47.558842: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64:/opt/conda/lib
2021-08-03 11:37:47.558918: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-03 11:37:49.562273: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-03 11:37:49.578957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-03 11:37:49.579987: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB comp

In [30]:
!python convert_tflite.py\
 --weights ./yolov4-tiny-pretflite-416 \
 --output ./yolov4-tiny-416.tflite \
#  --quantize_mode int8 \
#  --dataset ../../input/yolo-tflite-jackpodkim/test.txt

2021-08-03 11:38:13.280942: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64:/opt/conda/lib
2021-08-03 11:38:13.280998: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-03 11:38:15.327079: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-03 11:38:15.344983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-03 11:38:15.346068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB comp

---
# test

In [27]:
!python detect.py --weights ./yolov4-tiny-416 --size 416 --model yolov4 \
  --image ../../input/yolo-tflite-jackpodkim/test_img.jpg 

2021-08-03 11:03:05.540793: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64:/opt/conda/lib
2021-08-03 11:03:05.540904: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-03 11:03:07.761077: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-03 11:03:07.778604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-03 11:03:07.779712: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB comp

In [26]:
!python detect.py \
 --weights ./yolov4-tiny-416.tflite \
 --size 416 --model yolov4 \
 --image ../../input/yolo-tflite-jackpodkim/test_img.jpg \
 --framework tflite


2021-08-03 10:52:09.633806: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64:/opt/conda/lib
2021-08-03 10:52:09.633912: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-03 10:52:11.629072: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-03 10:52:11.646096: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-03 10:52:11.647209: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB comp